In [2]:
import time
import re
import pandas as pd
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
# Liste des liens

urls = ["https://emploi.educarriere.ci/nos-offres?page1={}&codes=&mots_cles=&typeemploi1=&niveau1=&anciennete=&typeoffre1=&recruteur=".format(category) for category in range(41)]

# En-tête pour éviter d'être bloqué
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}

def extract_text(element, class_name=None, style=None, text_contains=None):
    if element:
        tag = element.find(class_=class_name, style=style, text=text_contains)
        return tag.text.strip() if tag else ""
    else:
        return ""

def clean_text(text):
    if text is not None:
        cleaned_text = text.replace('D\x92', ' ').replace('d\x92', ' ').replace('\x92', ' ').replace('\r\n', '').replace('\xa0', '')
        return cleaned_text.strip() if cleaned_text else None
    else:
        return None

def extract_date(element, text_contains):
    date_elements = element.find_all('a', class_='text')
    date = next((e.find('span', style='color:#FF0000;').text.strip() for e in date_elements if text_contains in e.text), "")
    return date

def scrape_emploi_ci(url):
    try:
        response = requests.get(url, timeout=200)
        response.raise_for_status()
    except requests.exceptions.RequestException as e:
        print(f"Erreur de connexion à {url} : {e}")
        return pd.DataFrame()

    soup = BeautifulSoup(response.text, 'html.parser')

    job_description_wrappers = soup.find_all('div', class_='box row')

    data_list = []

    for wrapper in job_description_wrappers:
        h4_tag = wrapper.find('h4')
        poste = extract_text(h4_tag)

        entry_title_tag = wrapper.find('p', class_='entry-title')
        sous_titre = extract_text(entry_title_tag)

        a_text_tag = wrapper.find('a', class_='text')
        code = extract_text(a_text_tag, style='color:#FF0000;')

        date_edition = extract_date(wrapper, "Date d'édition:")
        date_limite = extract_date(wrapper, "Date limite:")

        pays_tag = wrapper.find('a', class_='text')
        pays = pays_tag.find_parent().text.strip().split()[-1] if pays_tag else None

        sous_titre = clean_text(sous_titre)

        data_list.append({
            'Poste': clean_text(poste),
            'Sous_titre': sous_titre,
            'Code': clean_text(code),
            'Date_DEdition': date_edition,
            'Date_limite': date_limite,
            'Pays': clean_text(pays)
        })

    df = pd.DataFrame(data_list)
    return df


# Créer un DataFrame à partir des liens
result_df = pd.concat([scrape_emploi_ci(url) for url in urls], ignore_index=True)

# Supprimer les lignes dont toutes les variables n'ont pas de données
result_df = result_df.dropna(how='any').reset_index(drop=True)

# Afficher le DataFrame
result_df

C:\Users\ngora\AppData\Local\Temp\ipykernel_17556\355123160.py:18: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  tag = element.find(class_=class_name, style=style, text=text_contains)


,Poste,Sous_titre,Code,Date_DEdition,Date_limite,Pays
0,Manager-Mine Technical Services_Cote d'Ivoire,Manager-Mine Technical Services_Cote d’IvoireÀ...,108936,19/01/2024,10/02/2024,d'I...
1,COMMERCIALE ASSURANCE,BAMADH&COrecruteCOMMERCIALE ASSURANCEDescripti...,108935,19/01/2024,29/01/2024,C...
2,DESSINATEUR-PROJETEUR,SDL HOLDINGrecruteDESSINATEUR-PROJETEURDescrip...,108934,19/01/2024,29/01/2024,d'I...
3,CHEF COMPTABLE,KAME SERVICES PLUSrecruteCHEF COMPTABLEDescrip...,108932,19/01/2024,19/02/2024,...
4,1 GESTIONNAIRE DE STOCK POUR CHAQUE SITE CI-DE...,Description du posteROSAPARKS recrute pour une...,108930,19/01/2024,22/01/2024,SASSANDRA...
5,Un (01) Coordonnateur en Communication et Gest...,DANS LE CADRE DU PROJET « STOP DJEKOIDJO» FINA...,108929,19/01/2024,26/01/2024,d'I...
6,"Chargé(e) de Suivi, Evaluation, Redevabilité e...","Chargé(e) de Suivi, Evaluation, Redevabilité e...",108928,19/01/2024,01/02/2024,d'I...
7,"Assistant Projet, sur ProGouvCI  Save The Chi...","Assistant Projet, sur ProGouvCI – Save The Chi...",108927,19/01/2024,01/02/2024,d'I...
8,COMMERCIAL,DOOYA GROUP SARLrecruteCOMMERCIALDescription d...,108926,19/01/2024,29/02/2024,KOUMASSI
9,PLOMBIER,ST2I & COrecrutePLOMBIERDescription du poste- ...,108925,19/01/2024,29/01/2024,Abidjan


In [3]:
options = webdriver.ChromeOptions()
options.add_argument("--headless")  # Pour exécuter le navigateur en arrière-plan
options.add_argument("--disable-gpu")  # Désactiver l'accélération GPU en mode headless
chrome_driver_path = "C:\\Users\\ngora\\OneDrive\\Bureau\\INS_DATA\\chromedriver_win32\\chromedriver.exe"
options.binary_location = "C:\\Program Files\\Google\\Chrome\\Application\\chrome.exe"  # Remplacez par l'emplacement réel de votre Chrome binary
options.add_argument(f"webdriver.chrome.driver={chrome_driver_path}")
driver = webdriver.Chrome(options=options)

# Liste pour stocker les détails de chaque emploi
all_job_details = []

# Parcourir les liens
for url in urls:
    req = requests.get(url, headers=headers)
    soup = BeautifulSoup(req.text, 'html.parser')
    time.sleep(5)  # Attendre 5 secondes avant la prochaine requête

    offres = soup.find_all('div', class_='box row')

    # Parcourir les offres d'emploi sur la page principale
    for offre in offres:
        # Trouver la balise <h4> dans la structure HTML pour extraire le lien
        offre_link_tag = offre.find('h4')

        # Vérifier si la balise <h4> a été trouvée
        if offre_link_tag:
            # Extraire le lien de l'attribut 'href'
            offre_link = offre_link_tag.find('a')['href']
            all_job_details.append({'Offre_Link': offre_link})

# Fermer le pilote Selenium à la fin
driver.quit()

# Concaténer tous les détails des emplois en un seul DataFrame
if all_job_details:
    all_job_details_df = pd.DataFrame(all_job_details)
    # Afficher le DataFrame
    #print(all_job_details_df)
else:
    print("Aucun détail d'offre d'emploi trouvé.")


In [4]:
def extract_job_information(soup):
    # Extraction des informations de l'offre d'emploi
    poste = soup.select_one('li.list-group-item:-soup-contains("Poste")').strong.next_sibling.strip()
    type_offre = soup.select_one('li.list-group-item:-soup-contains("Type d\'offre")').strong.next_sibling.strip()
    metiers = soup.select_one('li.list-group-item:-soup-contains("Métier(s):")').strong.next_sibling.strip()
    niveaux = soup.select_one('li.list-group-item:-soup-contains("Niveau(x):")').strong.next_sibling.strip()
    experience = soup.select_one('li.list-group-item:-soup-contains("Expérience:")').strong.next_sibling.strip()
    lieu = soup.select_one('li.list-group-item:-soup-contains("Lieu:")').strong.next_sibling.strip()
    
    # Extraction des dates de publication et de limite
    date_publication = soup.find('strong', string='Date de publication:').find_next('span').text.strip()
    date_limite = soup.find('strong', string='Date limite:').find_next('span').text.strip()
    
   # description = soup.select_one('div.text-col.post.small-post.col-md-9.col-xs-12 ul.list-group').text.strip()
    description = soup.select_one('div.entry-content').text.strip()

    return {
        "Poste": [poste],
        "Type d'offre": [type_offre],
        "Métier(s)": [metiers],
        "Niveau(x)": [niveaux],
        "Expérience": [experience],
        "Lieu": [lieu],
        "Date de publication": [date_publication],
        "Date limite": [date_limite],
        "Description": [description]
    }

# Liste des URLs à scraper
urls = list(all_job_details_df['Offre_Link'])

# Liste pour stocker les DataFrames
dfs = []

# Boucle sur chaque URL
for url in urls:
    # Envoyer une requête GET au site
    response = requests.get(url)

    # Vérifier si la requête a réussi (statut 200)
    if response.status_code == 200:
        # Analyser le contenu de la page avec BeautifulSoup
        soup = BeautifulSoup(response.text, 'html.parser')

        try:
            # Extract job information
            job_info = extract_job_information(soup)

            # Create DataFrame
            df = pd.DataFrame(job_info)

            # Ajouter le DataFrame à la liste
            dfs.append(df)
        except Exception as e:
            print(f"An error occurred: {e}")
    else:
        print(f"Échec de la requête pour l'URL {url}. Statut : {response.status_code}")

# Concaténer tous les DataFrames en un seul DataFrame
df_Educarriere = pd.concat(dfs, ignore_index=True)
df_Educarriere


,Poste,Type d'offre,Métier(s),Niveau(x),Expérience,Lieu,Date de publication,Date limite,Description
0,Manager-Mine Technical Services_Cote d'Ivoire,Emploi,Mines/Géologie/Pétrole,"BAC+3, BAC+4",10 ans,Côte d'Ivoire,19/01/2024,10/02/2024,Manager-Mine Technical Services_Cote d’Ivoire\...
1,COMMERCIALE ASSURANCE,Stage,"Assurance, Commerce/Ventes","Première, Terminale, BAC+2, BAC+3",1 ans,Abidjan Cocody Riviéra Abatta,19/01/2024,29/01/2024,BAMADH&COrecruteCOMMERCIALE ASSURANCE\n \n \nD...
2,DESSINATEUR-PROJETEUR,Emploi,"Infographie, Informatique, NTIC","BAC+2, BAC+3, BAC+4",,Côte d'Ivoire,19/01/2024,29/01/2024,SDL HOLDINGrecruteDESSINATEUR-PROJETEUR\n \n \...
3,CHEF COMPTABLE,Emploi,Finances/Comptabilité,BAC+4,4 ans,"ABIDJAN, MARCORY ZONE 4",19/01/2024,19/02/2024,KAME SERVICES PLUSrecruteCHEF COMPTABLE\n \n \...
4,1 GESTIONNAIRE DE STOCK POUR CHAQUE SITE CI-DE...,Emploi,Logistique/Transport,BAC+2,1 ans,"SASSANDRA, SOUBRE, DIVO, TABOU, GAGNOA",19/01/2024,22/01/2024,Description du posteROSAPARKS recrute pour une...
5,Un (01) Coordonnateur en Communication et Gest...,Emploi,"Communication, Gestion, Marketing, Sciences so...",BAC+4,3 ans,Côte d'Ivoire,19/01/2024,26/01/2024,DANS LE CADRE DU PROJET « STOP DJEKOIDJO » FIN...
6,"Chargé(e) de Suivi, Evaluation, Redevabilité e...",Emploi,"Sciences sociales, Statistiques","BAC+3, BAC+4, BAC+5",,Côte d'Ivoire,19/01/2024,01/02/2024,"Chargé(e) de Suivi, Evaluation, Redevabilité e..."
7,"Assistant Projet, sur ProGouvCI  Save The Chi...",Emploi,Sciences sociales,BAC+4,,Côte d'Ivoire,19/01/2024,01/02/2024,"Assistant Projet, sur ProGouvCI – Save The Chi..."
8,COMMERCIAL,Stage,Commerce/Ventes,BAC+2,,KOUMASSI,19/01/2024,29/02/2024,DOOYA GROUP SARLrecruteCOMMERCIAL\n \n \nDescr...
9,PLOMBIER,Emploi,Plomberie,BT,1 ans,Abidjan,19/01/2024,29/01/2024,ST2I & COrecrutePLOMBIER\n \n \nDescription du...


In [5]:
# Ajouter les listes existantes en tant que colonnes au DataFrame
df_Educarriere['Poste'] = list(result_df['Poste'])
df_Educarriere['Sous_titre'] = list(result_df['Sous_titre'])
df_Educarriere['Code'] = list(result_df['Code'])
df_Educarriere['Date_DEdition'] = list(result_df['Date_DEdition'])
df_Educarriere['Date_limite'] = list(result_df['Date_limite'])
df_Educarriere['Pays'] = list(result_df['Pays'])


# Réorganiser les colonnes selon vos besoins
df_Educarriere 


,Poste,Type d'offre,Métier(s),Niveau(x),Expérience,Lieu,Date de publication,Date limite,Description,Sous_titre,Code,Date_DEdition,Date_limite,Pays
0,Manager-Mine Technical Services_Cote d'Ivoire,Emploi,Mines/Géologie/Pétrole,"BAC+3, BAC+4",10 ans,Côte d'Ivoire,19/01/2024,10/02/2024,Manager-Mine Technical Services_Cote d’Ivoire\...,Manager-Mine Technical Services_Cote d’IvoireÀ...,108936,19/01/2024,10/02/2024,d'I...
1,COMMERCIALE ASSURANCE,Stage,"Assurance, Commerce/Ventes","Première, Terminale, BAC+2, BAC+3",1 ans,Abidjan Cocody Riviéra Abatta,19/01/2024,29/01/2024,BAMADH&COrecruteCOMMERCIALE ASSURANCE\n \n \nD...,BAMADH&COrecruteCOMMERCIALE ASSURANCEDescripti...,108935,19/01/2024,29/01/2024,C...
2,DESSINATEUR-PROJETEUR,Emploi,"Infographie, Informatique, NTIC","BAC+2, BAC+3, BAC+4",,Côte d'Ivoire,19/01/2024,29/01/2024,SDL HOLDINGrecruteDESSINATEUR-PROJETEUR\n \n \...,SDL HOLDINGrecruteDESSINATEUR-PROJETEURDescrip...,108934,19/01/2024,29/01/2024,d'I...
3,CHEF COMPTABLE,Emploi,Finances/Comptabilité,BAC+4,4 ans,"ABIDJAN, MARCORY ZONE 4",19/01/2024,19/02/2024,KAME SERVICES PLUSrecruteCHEF COMPTABLE\n \n \...,KAME SERVICES PLUSrecruteCHEF COMPTABLEDescrip...,108932,19/01/2024,19/02/2024,...
4,1 GESTIONNAIRE DE STOCK POUR CHAQUE SITE CI-DE...,Emploi,Logistique/Transport,BAC+2,1 ans,"SASSANDRA, SOUBRE, DIVO, TABOU, GAGNOA",19/01/2024,22/01/2024,Description du posteROSAPARKS recrute pour une...,Description du posteROSAPARKS recrute pour une...,108930,19/01/2024,22/01/2024,SASSANDRA...
5,Un (01) Coordonnateur en Communication et Gest...,Emploi,"Communication, Gestion, Marketing, Sciences so...",BAC+4,3 ans,Côte d'Ivoire,19/01/2024,26/01/2024,DANS LE CADRE DU PROJET « STOP DJEKOIDJO » FIN...,DANS LE CADRE DU PROJET « STOP DJEKOIDJO» FINA...,108929,19/01/2024,26/01/2024,d'I...
6,"Chargé(e) de Suivi, Evaluation, Redevabilité e...",Emploi,"Sciences sociales, Statistiques","BAC+3, BAC+4, BAC+5",,Côte d'Ivoire,19/01/2024,01/02/2024,"Chargé(e) de Suivi, Evaluation, Redevabilité e...","Chargé(e) de Suivi, Evaluation, Redevabilité e...",108928,19/01/2024,01/02/2024,d'I...
7,"Assistant Projet, sur ProGouvCI  Save The Chi...",Emploi,Sciences sociales,BAC+4,,Côte d'Ivoire,19/01/2024,01/02/2024,"Assistant Projet, sur ProGouvCI – Save The Chi...","Assistant Projet, sur ProGouvCI – Save The Chi...",108927,19/01/2024,01/02/2024,d'I...
8,COMMERCIAL,Stage,Commerce/Ventes,BAC+2,,KOUMASSI,19/01/2024,29/02/2024,DOOYA GROUP SARLrecruteCOMMERCIAL\n \n \nDescr...,DOOYA GROUP SARLrecruteCOMMERCIALDescription d...,108926,19/01/2024,29/02/2024,KOUMASSI
9,PLOMBIER,Emploi,Plomberie,BT,1 ans,Abidjan,19/01/2024,29/01/2024,ST2I & COrecrutePLOMBIER\n \n \nDescription du...,ST2I & COrecrutePLOMBIERDescription du poste- ...,108925,19/01/2024,29/01/2024,Abidjan
